In [1]:
import os
import pandas as pd
import numpy as np
import pickle

import yaml

cfg = None
    #' load config.yaml file in the root dir 
with open("../config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)
    
# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

pd.set_option('display.float_format', lambda x: '%.2f' % x)


#Function to get data frame summary
def get_data_frame_summmary(data_frame, sort_by='percent_missing', ascending=False):
    unique_values = data_frame.apply(lambda x: [x.unique()])
    unique_counts = data_frame.apply(lambda x: len(x.unique()))
    percent_missing = data_frame.apply(lambda x: sum(pd.isnull(x))/len(x)*100)
    data_type = data_frame.dtypes

    return pd.DataFrame(dict(unique_values = unique_values, 
                                unique_counts = unique_counts,
                                data_type = data_type,
                                percent_missing = percent_missing,
                                )).reset_index().sort_values(by=sort_by, ascending=ascending)

# Function to drop outliers of numeric columns
def drop_outliers(data_frame, exclude=[]):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    
    for colname in num_cols:
        upper_lim = data_frame[colname].quantile(.95)
        lower_lim = data_frame[colname].quantile(.05)
        
        print(f'Dropping outliers for {colname} upper limit = {upper_lim} and lower limit = {lower_lim}')
        data_frame = data_frame[(data_frame[colname] < upper_lim) & (data_frame[colname] > lower_lim)]
      
    return data_frame


# Function to drop outliers of numeric columns
def scale_numeric_features(data_frame, exclude=[], 
                           method='standardize',
                           inplace=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    print(f'********************* - Scaling following {len(num_cols)} features - **********************')
    for colname in num_cols:
        new_colname = colname if inplace else colname+'_'+method+'d'
        print(f' {colname} {method}d to {new_colname}')
        
        if method == 'standardize':              
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].mean()) / data_frame[colname].std()
        elif method == 'normalize':
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].min()) / (data_frame[colname].max() - data_frame[colname].min())
        else:
              print(f'Unknown method {method} specified, please select one of "standardize" or "normalize"')

      
    return data_frame


def remove_items(iteamlist, removelist):
    return [ele for ele in iteamlist if ele not in removelist]

def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]
    
def months_between_columns(date_series1, date_series2):
    return round((date_series1-date_series2) / np.timedelta64(1, 'M'))
    


In [2]:
# Read Data
nps = pd.read_pickle('../data/pickle/NPS_Cleansed_Data.pickle')

df = nps
del nps

df.head()

,ECR_ID,RESPONSEID,WAVE,DATE_OF_INTERVIEW,ORG_NAME,COUNTRY,ORGANIZATION,PRODUCT_NAME_ROLLUP,PRODUCT_DETAIL,JOB_ROLE,...,COMPETITOR_NAME,DEPARTMENT,INFLUENCE,CSAT,CSAT_COMMENT,NPS_SCORE,NPS_COMMENT,AT_RISK,VALUE_FOR_MONEY_SCORE,SHARE_WITH_CUST_DETAILS
0,ECR-635,1362709,42.00,2015-01-29,Cornell University,United States,University,NaN,NaN,Other,...,NaN,Sales,No involvement,Satisfied,NaN,0.00,prefer not to answer because I donot usually m...,At Risk - Normal,NaN,NO
1,ECR-0,1378062,42.00,2015-02-26,NaN,Germany,University,NaN,NaN,Librarian,...,NaN,Online Customer Services,NaN,Very satisfied,No comment,0.00,pricing policy,At Risk - Normal,NaN,NO
2,ECR-0,1380278,42.00,2015-02-22,NaN,France,College,NaN,NaN,Student,...,NaN,Online Customer Services,NaN,Dissatisfied,"""Je pense que vous devez changer complÃ©tement...",0.00,"My previous message explains it clearly, the p...",At Risk - Red Flag,NaN,YES
3,ECR-1080234,1381630,42.00,2015-03-02,Miltenyi Biotec,United States,Commercial/Corporate,NaN,NaN,Other,...,NaN,Sales,Make recommendation,Neutral,NaN,0.00,Have not been contacted as part of a follow up...,At Risk - Normal,Don't know/ NA,YES
4,ECR-0,1394468,42.00,2015-03-14,NaN,France,University,NaN,NaN,Researcher/scientist,...,NaN,Online Customer Services,NaN,Don't know,No comment,0.00,no answer,At Risk - Normal,NaN,YES


# Replace strings indicating missing data with null

In [3]:
#replace strings indicating missing data with null
df.replace(['nan', 'N.A', 'N.A.', 'NaN', 'Nan', '00-00-00', '0-00-00', 'Unknown', 'No comment', 'No', 'no', 'no comment', "Don't know/ NA", "Don't know"], np.nan, inplace=True)
df.replace('.*\?.*', np.nan, regex=True, inplace=True)
df

,ECR_ID,RESPONSEID,WAVE,DATE_OF_INTERVIEW,ORG_NAME,COUNTRY,ORGANIZATION,PRODUCT_NAME_ROLLUP,PRODUCT_DETAIL,JOB_ROLE,...,COMPETITOR_NAME,DEPARTMENT,INFLUENCE,CSAT,CSAT_COMMENT,NPS_SCORE,NPS_COMMENT,AT_RISK,VALUE_FOR_MONEY_SCORE,SHARE_WITH_CUST_DETAILS
0,ECR-635,1362709,42.00,2015-01-29,Cornell University,United States,University,NaN,NaN,Other,...,NaN,Sales,No involvement,Satisfied,NaN,0.00,NaN,At Risk - Normal,NaN,NO
1,ECR-0,1378062,42.00,2015-02-26,NaN,Germany,University,NaN,NaN,Librarian,...,NaN,Online Customer Services,NaN,Very satisfied,NaN,0.00,pricing policy,At Risk - Normal,NaN,NO
2,ECR-0,1380278,42.00,2015-02-22,NaN,France,College,NaN,NaN,Student,...,NaN,Online Customer Services,NaN,Dissatisfied,NaN,0.00,"My previous message explains it clearly, the p...",At Risk - Red Flag,NaN,YES
3,ECR-1080234,1381630,42.00,2015-03-02,Miltenyi Biotec,United States,Commercial/Corporate,NaN,NaN,Other,...,NaN,Sales,Make recommendation,Neutral,NaN,0.00,Have not been contacted as part of a follow up...,At Risk - Normal,NaN,YES
4,ECR-0,1394468,42.00,2015-03-14,NaN,France,University,NaN,NaN,Researcher/scientist,...,NaN,Online Customer Services,NaN,NaN,NaN,0.00,no answer,At Risk - Normal,NaN,YES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50526,ECR-215054,1355803,45.00,2015-12-02,Centre Hospitalier Chubert,NaN,NaN,NaN,NaN,NaN,...,NaN,e-Delivery,NaN,NaN,NaN,nan,NaN,At Risk - Normal,NaN,YES
50527,ECR-2228,1465409,nan,2015-12-02,Universitat De Barcelona,NaN,NaN,NaN,NaN,NaN,...,NaN,Sales,NaN,NaN,NaN,nan,NaN,At Risk - Normal,NaN,YES
50528,ECR-955455,1466318,45.00,2015-12-02,Wyzsza Szkola Nauk Spolecznych i Technicznych ...,NaN,NaN,NaN,NaN,NaN,...,NaN,e-Delivery,NaN,NaN,NaN,nan,NaN,At Risk - Normal,NaN,YES
50529,ECR-2778,1469519,nan,2015-12-02,Universitet I Oslo,NaN,NaN,NaN,NaN,NaN,...,NaN,Sales,NaN,NaN,NaN,nan,NaN,At Risk - Normal,NaN,YES


In [4]:
for i in range(0,df.shape[1]):
    print(df.iloc[:,i].value_counts())

ECR-0           35511
ECR-26877          86
ECR-1628           78
ECR-335922         66
ECR-339079         44
                ...  
ECR-765414          1
ECR-35171           1
ECR-11118611        1
ECR-353778          1
ECR-1137100         1
Name: ECR_ID, Length: 8464, dtype: int64
2099199    1
1697193    1
1751851    1
2209187    1
2207136    1
          ..
2189126    1
2193220    1
2180930    1
2187073    1
2230273    1
Name: RESPONSEID, Length: 50531, dtype: int64
58.00    5110
60.00    4809
59.00    4312
57.00    4226
54.00    3113
56.00    2649
53.00    2571
51.00    2278
46.00    2104
50.00    2033
52.00    1985
47.00    1958
49.00    1941
55.00    1851
48.00    1777
61.00    1432
45.00    1271
43.00    1263
42.00    1111
44.00     856
Name: WAVE, dtype: int64
2016-03-03    277
2018-09-27    233
2019-04-04    200
2019-07-25    186
2018-11-01    164
             ... 
2015-02-15      1
2017-07-07      1
2017-08-26      1
2016-11-12      1
2018-08-14      1
Name: DATE_OF_INTERVIEW, 

### Summary of the dataframe

In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
13,COMPETITOR_NAME,"[[nan, I have not dealt with any of these orga...",3816,object,82.29
21,VALUE_FOR_MONEY_SCORE,"[[nan, Neither agree nor disagree, Strongly ag...",6,object,72.48
17,CSAT_COMMENT,"[[nan, Provide better customer service trainin...",15780,object,68.22
12,COMP_RECOMMEND,"[[nan, 4.0, 0.0, 5.0, 10.0, 98.0, 9.0, 2.0, 7....",13,float64,55.65
19,NPS_COMMENT,"[[nan, pricing policy, My previous message exp...",21992,object,53.69
4,ORG_NAME,"[[Cornell University, nan, Miltenyi Biotec, Pr...",12262,object,52.15
11,COMPETITOR_SAT,"[[nan, Extremely dissatisfied, Very dissatisfi...",8,object,47.58
8,PRODUCT_DETAIL,"[[nan, Mendeley, ScienceDirect, Scopus, Clinic...",709,object,45.04
7,PRODUCT_NAME_ROLLUP,"[[nan, Mendeley, ScienceDirect, Scopus, Clinic...",73,object,44.92
15,INFLUENCE,"[[No involvement, nan, Make recommendation, Pr...",5,object,24.96


# Identify Missing Data

Missing values affect the performance of the machine learning models and its useful to identify and drop or impute missing values before modelling.

We will quantify the missing data and drop any columns lower than the given threshold. we have set threshold of 70% so any columns and rows with missing data over 70% are dropped.

In this case, manually ignore CSAT_COMMENT, NPS_COMMENT and PRODUCT_DETAIL

In [6]:
#set threshold
threshold = 0.7


#Dropping columns with missing value rate higher than threshold
temp = df[df.columns[df.isnull().mean() < threshold]]

print('-------------------- Dropping Columns with missing data --------------------')
print(f'Following {len(df.columns.difference(temp.columns))} columns have missing data over the threshold and will be removed')
print(df.columns.difference(temp.columns))

df = temp

print('-------------------- Dropping rows with missing data --------------------')
print(f' There are {sum(df.isnull().mean(axis=1) > threshold)} rows with missing data over the threshold')

#Dropping rows with missing value rate higher than threshold
df = df.loc[df.isnull().mean(axis=1) < threshold]

# remove temp object from memory
del temp

#manually remove borderline case
df = df.drop(columns=['CSAT_COMMENT', 'NPS_COMMENT', 'PRODUCT_DETAIL'])

-------------------- Dropping Columns with missing data --------------------
Following 2 columns have missing data over the threshold and will be removed
Index(['COMPETITOR_NAME', 'VALUE_FOR_MONEY_SCORE'], dtype='object')
-------------------- Dropping rows with missing data --------------------
 There are 0 rows with missing data over the threshold


In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
11,COMP_RECOMMEND,"[[nan, 4.0, 0.0, 5.0, 10.0, 98.0, 9.0, 2.0, 7....",13,float64,55.65
4,ORG_NAME,"[[Cornell University, nan, Miltenyi Biotec, Pr...",12262,object,52.15
10,COMPETITOR_SAT,"[[nan, Extremely dissatisfied, Very dissatisfi...",8,object,47.58
7,PRODUCT_NAME_ROLLUP,"[[nan, Mendeley, ScienceDirect, Scopus, Clinic...",73,object,44.92
13,INFLUENCE,"[[No involvement, nan, Make recommendation, Pr...",5,object,24.96
15,NPS_SCORE,"[[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0,...",12,float64,7.08
2,WAVE,"[[42.0, 43.0, 44.0, 45.0, nan, 46.0, 47.0, 48....",21,float64,3.72
14,CSAT,"[[Satisfied, Very satisfied, Dissatisfied, Neu...",8,object,2.35
8,JOB_ROLE,"[[Other, Librarian, Student, Researcher/scient...",2930,object,1.18
6,ORGANIZATION,"[[University, College, Commercial/Corporate, R...",1032,object,0.79


### Data Imputation

We can impute missing data with meaningful data so that model development has good quality of data

We can impute categorical variables with the most frequestly occuring value and impute numerical variables with 0 or mean or median depending on the variable context.

COMP_RECOMMEND - Impute with median

ORG_NAME - Impute with most frequent

COMPETITOR_SAT - Impute with most freuqent

PRODUCT_NAME_ROLLUP - Impute with most frequent

INFLUENCE - Impute with most frequent

NPS_SCORE - Impute with median

WAVE - Impute with median

CSAT - Impute with most frequent

JOB_ROLE - Impute with most frequent

In [8]:
# #Filling all missing values with 0
# # data = data.fillna(0)

# #Filling missing values with the most frequest values
# df['ORG_NAME'].fillna(
#     df['ORG_NAME'].value_counts().idxmax(), inplace=True)

# df['COMPETITOR_SAT'].fillna(
#     df['COMPETITOR_SAT'].value_counts().idxmax(), inplace=True)

# df['PRODUCT_NAME_ROLLUP'].fillna(
#     df['PRODUCT_NAME_ROLLUP'].value_counts().idxmax(), inplace=True)

# df['INFLUENCE'].fillna(
#     df['INFLUENCE'].value_counts().idxmax(), inplace=True)

# df['CSAT'].fillna(
#     df['CSAT'].value_counts().idxmax(), inplace=True)

# df['JOB_ROLE'].fillna(
#     df['JOB_ROLE'].value_counts().idxmax(), inplace=True)

# # df['COMP_RECOMMEND'].fillna(
# #     df['COMP_RECOMMEND'].median(), inplace=True)

# # df['NPS_SCORE'].fillna(
# #     df['NPS_SCORE'].median(), inplace=True)

# df['WAVE'].fillna(
#     df['WAVE'].median(), inplace=True)

# Handling Outliers

Extreme value can skew the data distribution and thus affect the model development we identify outliers in numeric variables and handle them by removing or capping.

### Outlier Detection with Percentiles

In [9]:
#Dropping the outlier rows with Percentiles

df = drop_outliers(df, exclude = ['NPS_SCORE', 'COMP_RECOMMEND', 'RESPONSEID', 'WAVE'])

# Binning

The main motivation of binning is to make the model more robust and prevent overfitting, however, it has a cost to the performance. Every time you bin something, you sacrifice information and make your data more regularized

The trade-off between performance and overfitting is the key point of the binning process


For Categorical variables the labels with low frequencies probably affect the robustness of statistical models negatively. Thus, assigning a general category to these less frequent values helps to keep the robustness of the model.

it is a good option to unite the labels with a count less than 100 to a new category like “Other”.

In [10]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)   

### Columns with less than 100 unique categories

In [11]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])
    

,index,unique_values,unique_counts,data_type,percent_missing
17,SHARE_WITH_CUST_DETAILS,"[[NO, YES]]",2,object,0.00
16,AT_RISK,"[[At Risk - Normal, At Risk - Red Flag, At Ris...",5,object,0.73
13,INFLUENCE,"[[No involvement, nan, Make recommendation, Pr...",5,object,24.96
14,CSAT,"[[Satisfied, Very satisfied, Dissatisfied, Neu...",8,object,2.35
10,COMPETITOR_SAT,"[[nan, Extremely dissatisfied, Very dissatisfi...",8,object,47.58
12,DEPARTMENT,"[[Sales, Online Customer Services, e-Delivery,...",11,object,0.00
15,NPS_SCORE,"[[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0,...",12,float64,7.08
11,COMP_RECOMMEND,"[[nan, 4.0, 0.0, 5.0, 10.0, 98.0, 9.0, 2.0, 7....",13,float64,55.65
2,WAVE,"[[42.0, 43.0, 44.0, 45.0, nan, 46.0, 47.0, 48....",21,float64,3.72
9,JOB_ROLE_GROUPED,"[[Other, Librarian, Student, Researcher/scient...",23,object,0.00


In [12]:
columns_to_bin = binning_summary[binning_summary['unique_counts'] < 100]['index'].to_list()

In [13]:
# select columns with less than 100 labels
df = df.apply(lambda x: x.mask(x.map(x.value_counts())<10, 'Other') if x.name in columns_to_bin else x)

### After Binning - Columns with less than 100 unique categories

In [14]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])

,index,unique_values,unique_counts,data_type,percent_missing
17,SHARE_WITH_CUST_DETAILS,"[[NO, YES]]",2,object,0.00
16,AT_RISK,"[[At Risk - Normal, At Risk - Red Flag, At Ris...",5,object,0.73
13,INFLUENCE,"[[No involvement, nan, Make recommendation, Pr...",5,object,24.96
14,CSAT,"[[Satisfied, Very satisfied, Dissatisfied, Neu...",8,object,2.35
10,COMPETITOR_SAT,"[[nan, Extremely dissatisfied, Very dissatisfi...",8,object,47.58
12,DEPARTMENT,"[[Sales, Online Customer Services, e-Delivery,...",11,object,0.00
15,NPS_SCORE,"[[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0,...",12,float64,7.08
11,COMP_RECOMMEND,"[[nan, 4.0, 0.0, 5.0, 10.0, 98.0, 9.0, 2.0, 7....",13,float64,55.65
2,WAVE,"[[42.0, 43.0, 44.0, 45.0, nan, 46.0, 47.0, 48....",21,float64,3.72
9,JOB_ROLE_GROUPED,"[[Other, Librarian, Student, Researcher/scient...",22,object,0.00


## Log Transform

Logarithm transformation (or log transform) is one of the most commonly used mathematical transformations

It helps to handle skewed data and after transformation, the distribution becomes more approximate to normal.
In most of the cases the magnitude order of the data changes within the range of the data. log transform normalizes the magnitude differences.

It also decreases the effect of the outliers, due to the normalization of magnitude differences and the model become more robust.

Important note: The data you apply log transform must have only positive values, otherwise you receive an error. Also, you can add 1 to your data before transform it. Thus, you ensure the output of the transformation to be positive.


In [15]:
# No Log transform applied 

## Scaling

In most cases, the numerical features of the dataset do not have a certain range and they differ from each other. Scaling solves this problem. The continuous features become identical in terms of the range, after a scaling process. This process is not mandatory for many algorithms, but it might be still nice to apply. 

### Normalization

Normalization (or min-max normalization) scale all values in a fixed range between 0 and 1. This transformation does not change the distribution of the feature and due to the decreased standard deviations, the effects of the outliers increases. Therefore, before normalization, it is recommended to handle the outliers

### Standardization
Standardization (or z-score normalization) scales the values while taking into account standard deviation. If the standard deviation of features is different, their range also would differ from each other. This reduces the effect of the outliers in the features.
In the following formula of standardization, the mean is shown as μ and the standard deviation is shown as σ.

In [16]:
#df = scale_numeric_features(df, inplace=True)

## One-hot encoding

This method spreads the values in a column to multiple flag columns and assigns 0 or 1 to them. These binary values express the relationship between grouped and encoded column.

This method changes your categorical data, which is challenging to understand for algorithms, to a numerical format

If you have N distinct values in the column, it is enough to map them to N-1, as the missing value can be deduced from the other columns



In [17]:
#print(columns_to_bin)

In [18]:
# # Specify Columns to encode
# columns_to_exclude = []
# columns_to_include = columns_to_bin
# columns_to_encode = remove_items(columns_to_include, columns_to_exclude)
# columns_to_encode

In [19]:
# for column in columns_to_encode:
#     encoded_columns = pd.get_dummies(df[column])
#     print(f'Encoding collumns : {column} to {len(encoded_columns.columns)} new encoded columns')
#     df = df.join(encoded_columns, rsuffix='_'+column).drop(column, axis=1)

In [20]:
# df.shape

In [21]:
df.head()

,ECR_ID,RESPONSEID,WAVE,DATE_OF_INTERVIEW,ORG_NAME,COUNTRY,ORGANIZATION,PRODUCT_NAME_ROLLUP,JOB_ROLE,JOB_ROLE_GROUPED,COMPETITOR_SAT,COMP_RECOMMEND,DEPARTMENT,INFLUENCE,CSAT,NPS_SCORE,AT_RISK,SHARE_WITH_CUST_DETAILS
0,ECR-635,1362709,42.00,2015-01-29,Cornell University,United States,University,NaN,Other,Other,NaN,nan,Sales,No involvement,Satisfied,0.00,At Risk - Normal,NO
1,ECR-0,1378062,42.00,2015-02-26,NaN,Germany,University,NaN,Librarian,Librarian,NaN,nan,Online Customer Services,NaN,Very satisfied,0.00,At Risk - Normal,NO
2,ECR-0,1380278,42.00,2015-02-22,NaN,France,College,NaN,Student,Student,NaN,nan,Online Customer Services,NaN,Dissatisfied,0.00,At Risk - Red Flag,YES
3,ECR-1080234,1381630,42.00,2015-03-02,Miltenyi Biotec,United States,Commercial/Corporate,NaN,Other,Other,NaN,nan,Sales,Make recommendation,Neutral,0.00,At Risk - Normal,YES
4,ECR-0,1394468,42.00,2015-03-14,NaN,France,University,NaN,Researcher/scientist,Researcher/scientist,NaN,nan,Online Customer Services,NaN,NaN,0.00,At Risk - Normal,YES


## Transform Date columns

Recency and Frequency Variables and length / duration.


In [22]:
# convert date columns to date data type

date_columns = ['DATE_OF_INTERVIEW']

In [23]:
for column in date_columns:
    df[column] = pd.to_datetime(df[column])

In [24]:
# calcualte new variable subscription_length and agreement_length

In [25]:
 df.head()

,ECR_ID,RESPONSEID,WAVE,DATE_OF_INTERVIEW,ORG_NAME,COUNTRY,ORGANIZATION,PRODUCT_NAME_ROLLUP,JOB_ROLE,JOB_ROLE_GROUPED,COMPETITOR_SAT,COMP_RECOMMEND,DEPARTMENT,INFLUENCE,CSAT,NPS_SCORE,AT_RISK,SHARE_WITH_CUST_DETAILS
0,ECR-635,1362709,42.00,2015-01-29,Cornell University,United States,University,NaN,Other,Other,NaN,nan,Sales,No involvement,Satisfied,0.00,At Risk - Normal,NO
1,ECR-0,1378062,42.00,2015-02-26,NaN,Germany,University,NaN,Librarian,Librarian,NaN,nan,Online Customer Services,NaN,Very satisfied,0.00,At Risk - Normal,NO
2,ECR-0,1380278,42.00,2015-02-22,NaN,France,College,NaN,Student,Student,NaN,nan,Online Customer Services,NaN,Dissatisfied,0.00,At Risk - Red Flag,YES
3,ECR-1080234,1381630,42.00,2015-03-02,Miltenyi Biotec,United States,Commercial/Corporate,NaN,Other,Other,NaN,nan,Sales,Make recommendation,Neutral,0.00,At Risk - Normal,YES
4,ECR-0,1394468,42.00,2015-03-14,NaN,France,University,NaN,Researcher/scientist,Researcher/scientist,NaN,nan,Online Customer Services,NaN,NaN,0.00,At Risk - Normal,YES


## Remove unused and/or redundant Features

Unused features are those that don’t make sense to pass into our machine learning algorithms. such as ID columns
Features that wouldn't be available at the time of prediction, Other text descriptions

Redundant features would typically be those that have been replaced by other features that you’ve added during feature engineering


In [26]:
columns_to_remove = ['DATE_OF_INTERVIEW', 'RESPONSEID', 'ORG_NAME']

In [27]:
df = df.drop(columns_to_remove, axis=1)

In [28]:
df.head()

,ECR_ID,WAVE,COUNTRY,ORGANIZATION,PRODUCT_NAME_ROLLUP,JOB_ROLE,JOB_ROLE_GROUPED,COMPETITOR_SAT,COMP_RECOMMEND,DEPARTMENT,INFLUENCE,CSAT,NPS_SCORE,AT_RISK,SHARE_WITH_CUST_DETAILS
0,ECR-635,42.00,United States,University,NaN,Other,Other,NaN,nan,Sales,No involvement,Satisfied,0.00,At Risk - Normal,NO
1,ECR-0,42.00,Germany,University,NaN,Librarian,Librarian,NaN,nan,Online Customer Services,NaN,Very satisfied,0.00,At Risk - Normal,NO
2,ECR-0,42.00,France,College,NaN,Student,Student,NaN,nan,Online Customer Services,NaN,Dissatisfied,0.00,At Risk - Red Flag,YES
3,ECR-1080234,42.00,United States,Commercial/Corporate,NaN,Other,Other,NaN,nan,Sales,Make recommendation,Neutral,0.00,At Risk - Normal,YES
4,ECR-0,42.00,France,University,NaN,Researcher/scientist,Researcher/scientist,NaN,nan,Online Customer Services,NaN,NaN,0.00,At Risk - Normal,YES


Save clean pickle files

In [29]:
df.to_pickle("../"+cfg['clean_pickles_for_graphs_dir']+"/"+cfg['nps_file']+"_for_graphs.pickle")